[View in Colaboratory](https://colab.research.google.com/github/tchappui/heia-cours-genie-chimique/blob/master/Chapitre1/CGC3_Exercice1.3_1.4.ipynb)

# Exercices 1.3 et 1.4




On commence par importer le solveur pour résoudre notre équation.

In [0]:
import math as m

import scipy.optimize as opt
import numpy as np
import matplotlib.pyplot as plt

On commence par rechercher les contantes sur NIST Webbook et définir les constantes comme par exemple la pression de notre système.

In [0]:
hexane = {
    'A': 4.00266,	
    'B': 1171.53,
    'C': -48.784
}

ethanol = {
    'A': 5.24677,
    'B': 1598.673,
    'C': -46.424
}


In [0]:
def antoine(produit, T):
    """Calcule la pression de vapeur saturante pour un produit
    donné en bar à une température T donnée en K.
    """
    log10_p = produit['A'] - produit['B'] / (T + produit['C'])
    return 10 ** log10_p


In [0]:
def raoult(produit, x, T):
    """Calcule la pression partielle d'un produit à l'aide de la loi de Raoult
    à une température T donnée en K et une fraction molaire x.
    """
    return x * antoine(produit, T)

In [0]:
def corrected_raoult(produit, x, T, gamma):
    return gamma * raoult(produit, x, T)


In [0]:
def compute_vanlaar_params(gammaA, gammaB, xA):
    xB = 1 - xA
    A = m.log(gammaA) * (1 + (xB * m.log(gammaB))/(xA * m.log(gammaA)))**2
    B = m.log(gammaB) * (1 + (xA * m.log(gammaA))/(xB * m.log(gammaB)))**2
    return A, B

In [0]:
def vanlaar(A, B, xA):
    xB = 1 - xA
    ln_gammaA = A / (1 + (xA * A) / (xB * B))**2
    ln_gammaB = B / (1 + (xB * B) / (xA * A))**2
    return m.e**ln_gammaA, m.e**ln_gammaB

On commencer par déterminer les coefficients d'activité de A et de B dans les conditions du point azéotropique, soit une fraction molaire de 33.2 mol% d'hexane et une température d'ébullition du mélange de 58.7 °C.

In [0]:
Psys = 1.01325 # bar (1 atm)
Tsys = 58.7 + 273.15 # (K)
xH_azeo = 0.332
xE_azeo = 1 - xH_azeo

Les pressions de vapeur saturantes selon Antoine sont:

In [9]:
PS_hexane = antoine(hexane, Tsys)
PS_hexane

0.7310435241055441

In [10]:
PS_ethanol = antoine(ethanol, Tsys)
PS_ethanol

0.44234699775114633

In [11]:
gamma_hexane = Psys / PS_hexane
gamma_hexane

1.386032386019348

In [12]:
gamma_ethanol = Psys / PS_ethanol
gamma_ethanol

2.2906225319743885

Nous allons calculer les paramètres de van Laar

In [0]:
A, B = compute_vanlaar_params(gamma_hexane, gamma_ethanol, xH_azeo)

In [14]:
gammaH, gammaE = vanlaar(A, B, 0.6)
(gammaH, gammaE)

(1.0462273603586822, 2.843698478867334)

Je veux calculer la température d'ébullition pour xA = xB = 0.5

In [0]:
def equation(Teb, prod1, prod2, xA, gammaA, gammaB):
    return corrected_raoult(prod1, xA, Teb, gammaA) + corrected_raoult(prod2, 1-xA, Teb, gammaB) - Psys



In [16]:
opt.fsolve(equation, 69+273, (hexane, ethanol, 0.6, gammaH, gammaE)) - 273.15


array([60.01137448])